# Pythonで沖縄電力の需給実績情報のデータを処理

## この環境の使い方


In [1]:
# 必要なモジュールをインポート
import pandas as pd #データ操作用モジュールを読み込み
# import numpy as np #数値計算用モジュールを読み込み
from datetime import date #日付処理用モジュールを読み込み

# 以下はグラフを描かなければ不要
from bokeh.plotting import figure, show, output_notebook #グラフ表示モジュールを読み込み
from bokeh.palettes import Category10 as palette #色のパレットを読み込み
from bokeh.palettes import Colorblind as paletteblind #色のパレットを読み込み（色弱用）
output_notebook() 


Loading BokehJS ...

In [2]:
# 沖縄電力の需給実績をダウンロードして左のファイル表示欄にドラッグ
# 電力のデータは MWh である
# filename変数の文字列をファイル名に合わせる
filename="2016.csv"
#
#
df_loadokinawa=pd.read_csv(filename,encoding='cp932',header=[5],skiprows=[6])
# 見出し行（ヘッダー）を整理
new_columns=["日付","時刻","需要実績","空列0","火力","水力","バイオマス","太陽光","太陽光制御","風力","風力制御","発電合計"]

# 年によって列数が違うので最後列に空の列がある場合に削除
df_loadokinawa.drop(df_loadokinawa.columns[range(len(new_columns),len(df_loadokinawa.columns.values))],
                    axis=1,inplace=True)
# 整理した見出し行に変更
df_loadokinawa.columns=new_columns
# 途中にある無駄な空の列を削除
df_loadokinawa.drop("空列0",axis=1,inplace=True)

# Pythonで扱い易いように文字列の日付情報と時刻情報から日時の列を生成
df_loadokinawa["日時"]= pd.to_datetime(df_loadokinawa["日付"]+"/"+df_loadokinawa["時刻"], format='%Y/%m/%d/%H:%M')
# 太陽光を除いた他の発電が担うべき正味需要を計算
df_loadokinawa["正味需要"]=df_loadokinawa["需要実績"]-df_loadokinawa["太陽光"]
# 日時情報から年月日だけ抜き出した列を生成（日単位の処理を簡単にするため）
df_loadokinawa["年月日"]=df_loadokinawa["日時"].dt.strftime("%Y-%m-%d").astype("datetime64")
# 日時情報から月だけ抜き出した列を生成（月単位の処理を簡単にするため）
df_loadokinawa["月"]=df_loadokinawa["日時"].dt.month
# 日時情報から曜日の列を生成（曜日単位の処理を簡単にするため）
df_loadokinawa["曜日"]=df_loadokinawa["日時"].dt.day_name()
df_loadokinawa["時刻"]=df_loadokinawa["日時"].dt.hour

In [3]:
df_loadokinawa

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,月,曜日
0,2016/4/1,0,647,643,1,2,0,0,2,0,647,2016-04-01 00:00:00,647,2016-04-01,4,Friday
1,2016/4/1,1,630,625,1,2,0,0,3,0,630,2016-04-01 01:00:00,630,2016-04-01,4,Friday
2,2016/4/1,2,644,640,1,2,0,0,2,0,644,2016-04-01 02:00:00,644,2016-04-01,4,Friday
3,2016/4/1,3,663,658,1,2,0,0,2,0,663,2016-04-01 03:00:00,663,2016-04-01,4,Friday
4,2016/4/1,4,657,652,1,2,0,0,3,0,657,2016-04-01 04:00:00,657,2016-04-01,4,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017/3/31,19,881,861,1,3,0,0,14,0,881,2017-03-31 19:00:00,881,2017-03-31,3,Friday
8756,2017/3/31,20,849,828,1,3,0,0,15,0,849,2017-03-31 20:00:00,849,2017-03-31,3,Friday
8757,2017/3/31,21,804,785,1,3,0,0,14,0,804,2017-03-31 21:00:00,804,2017-03-31,3,Friday
8758,2017/3/31,22,752,733,1,3,0,0,13,0,752,2017-03-31 22:00:00,752,2017-03-31,3,Friday


In [4]:
# 1日ごとの合計を出したければgroupbyメソッドで年月日のグループごとに取り出しsumで総和を取る
# ["需要実績","太陽光","正味需要"]のところにある列だけを抜き出して合計をとる
df_loadokinawa_daily_sum=df_loadokinawa.groupby("年月日")[["需要実績","太陽光","正味需要"]].sum()
df_loadokinawa_daily_sum

,需要実績,太陽光,正味需要
年月日,,,
2016-04-01,18328,1660,16668
2016-04-02,18560,1624,16936
2016-04-03,18148,1577,16571
2016-04-04,19692,535,19157
2016-04-05,20018,1036,18982
...,...,...,...
2017-03-27,18463,1879,16584
2017-03-28,18420,1901,16519
2017-03-29,18173,1313,16860


In [24]:
# Excelで読めるようにCSV形式で保存することが可能
# Excelに読み込ませないのであれば encoding='cp932' は消す
# 
df_loadokinawa_daily_sum.to_csv("loadokinawa_daily_sum.csv",encoding='cp932')
#
# Excelに読み込ませないのであれば下の行頭の#を削除して使う
# df_loadokinawa_daily_sum.to_csv("loadokinawa_daily_sum.csv")


In [5]:
# 1日の最大値を出したければgroupbyメソッドで年月日のグループごとに取り出しmaxを渡す
df_loadokinawa_daily_max=df_loadokinawa.groupby("年月日")[["需要実績","太陽光","正味需要"]].max()
df_loadokinawa_daily_max

,需要実績,太陽光,正味需要
年月日,,,
2016-04-01,863,230,863
2016-04-02,884,233,884
2016-04-03,890,223,890
2016-04-04,945,79,920
2016-04-05,963,154,957
...,...,...,...
2017-03-27,864,256,864
2017-03-28,867,253,867
2017-03-29,863,234,863


In [6]:
# 月ごとも同様に
df_loadokinawa_monthly_sum=df_loadokinawa.groupby("月")[["需要実績","太陽光","正味需要"]].sum()
df_loadokinawa_monthly_sum

,需要実績,太陽光,正味需要
月,,,
1,576422,20906,555516
2,535876,24281,511595
3,578989,29821,549168
4,602059,35916,566143
5,697359,37867,659492
6,796932,37331,759601
7,884861,42364,842497
8,869261,40489,828772
9,788481,33091,755390


In [7]:
# 月ごとの合計から年間の合計を算出
df_loadokinawa_monthly_sum.sum()

需要実績    8285756
太陽光      387009
正味需要    7898747
dtype: int64

以下は参考

In [8]:
# 特定の月のデータだけ出してみる
selected_month=8
df_loadokinawa.query('月 == @selected_month').head(10) #　.head(10)で最初の10行に制限

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,月,曜日
2928,2016/8/1,0,1008,1005,1,2,0,0,0,0,1008,2016-08-01 00:00:00,1008,2016-08-01,8,Monday
2929,2016/8/1,1,946,943,1,2,0,0,0,0,946,2016-08-01 01:00:00,946,2016-08-01,8,Monday
2930,2016/8/1,2,933,930,1,2,0,0,0,0,933,2016-08-01 02:00:00,933,2016-08-01,8,Monday
2931,2016/8/1,3,925,922,1,2,0,0,0,0,925,2016-08-01 03:00:00,925,2016-08-01,8,Monday
2932,2016/8/1,4,916,913,1,2,0,0,0,0,916,2016-08-01 04:00:00,916,2016-08-01,8,Monday
2933,2016/8/1,5,927,924,1,2,0,0,0,0,927,2016-08-01 05:00:00,927,2016-08-01,8,Monday
2934,2016/8/1,6,955,938,1,2,14,0,0,0,955,2016-08-01 06:00:00,941,2016-08-01,8,Monday
2935,2016/8/1,7,1073,1012,1,2,58,0,0,0,1073,2016-08-01 07:00:00,1015,2016-08-01,8,Monday
2936,2016/8/1,8,1206,1122,1,2,81,0,0,0,1206,2016-08-01 08:00:00,1125,2016-08-01,8,Monday
2937,2016/8/1,9,1332,1188,1,2,141,0,0,0,1332,2016-08-01 09:00:00,1191,2016-08-01,8,Monday


In [9]:
# 特定の日のデータだけ出してみる
selected_date="2016-08-05"
df_loadokinawa.query('年月日==@selected_date')

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,月,曜日
3024,2016/8/5,0,959,956,1,2,0,0,0,0,959,2016-08-05 00:00:00,959,2016-08-05,8,Friday
3025,2016/8/5,1,902,899,1,2,0,0,0,0,902,2016-08-05 01:00:00,902,2016-08-05,8,Friday
3026,2016/8/5,2,887,885,1,2,0,0,0,0,887,2016-08-05 02:00:00,887,2016-08-05,8,Friday
3027,2016/8/5,3,879,876,1,2,0,0,0,0,879,2016-08-05 03:00:00,879,2016-08-05,8,Friday
3028,2016/8/5,4,865,863,1,2,0,0,0,0,865,2016-08-05 04:00:00,865,2016-08-05,8,Friday
3029,2016/8/5,5,870,867,1,2,0,0,0,0,870,2016-08-05 05:00:00,870,2016-08-05,8,Friday
3030,2016/8/5,6,891,879,1,2,9,0,1,0,891,2016-08-05 06:00:00,882,2016-08-05,8,Friday
3031,2016/8/5,7,981,959,1,2,19,0,0,0,981,2016-08-05 07:00:00,962,2016-08-05,8,Friday
3032,2016/8/5,8,1100,1069,1,2,28,0,0,0,1100,2016-08-05 08:00:00,1072,2016-08-05,8,Friday
3033,2016/8/5,9,1188,1120,1,2,66,0,0,0,1188,2016-08-05 09:00:00,1122,2016-08-05,8,Friday


In [10]:
# 特定の範囲のデータを出す
start_date="2016-08-05"
end_date="2016-08-06"
df_loadokinawa.query('@start_date <= 年月日 <= @end_date')

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,月,曜日
3024,2016/8/5,0,959,956,1,2,0,0,0,0,959,2016-08-05 00:00:00,959,2016-08-05,8,Friday
3025,2016/8/5,1,902,899,1,2,0,0,0,0,902,2016-08-05 01:00:00,902,2016-08-05,8,Friday
3026,2016/8/5,2,887,885,1,2,0,0,0,0,887,2016-08-05 02:00:00,887,2016-08-05,8,Friday
3027,2016/8/5,3,879,876,1,2,0,0,0,0,879,2016-08-05 03:00:00,879,2016-08-05,8,Friday
3028,2016/8/5,4,865,863,1,2,0,0,0,0,865,2016-08-05 04:00:00,865,2016-08-05,8,Friday
3029,2016/8/5,5,870,867,1,2,0,0,0,0,870,2016-08-05 05:00:00,870,2016-08-05,8,Friday
3030,2016/8/5,6,891,879,1,2,9,0,1,0,891,2016-08-05 06:00:00,882,2016-08-05,8,Friday
3031,2016/8/5,7,981,959,1,2,19,0,0,0,981,2016-08-05 07:00:00,962,2016-08-05,8,Friday
3032,2016/8/5,8,1100,1069,1,2,28,0,0,0,1100,2016-08-05 08:00:00,1072,2016-08-05,8,Friday
3033,2016/8/5,9,1188,1120,1,2,66,0,0,0,1188,2016-08-05 09:00:00,1122,2016-08-05,8,Friday


In [15]:
# 特定の範囲の日付の特定の時間のみ
start_date="2016-08-05"
end_date="2016-08-06"
start_time=8
end_time=18
df_loadokinawa.query('@start_date <= 年月日 <= @end_date and @start_time <= 時刻 <= @end_time')

,日付,時刻,需要実績,火力,水力,バイオマス,太陽光,太陽光制御,風力,風力制御,発電合計,日時,正味需要,年月日,月,曜日
3032,2016/8/5,8,1100,1069,1,2,28,0,0,0,1100,2016-08-05 08:00:00,1072,2016-08-05,8,Friday
3033,2016/8/5,9,1188,1120,1,2,66,0,0,0,1188,2016-08-05 09:00:00,1122,2016-08-05,8,Friday
3034,2016/8/5,10,1221,1111,1,2,107,0,0,0,1221,2016-08-05 10:00:00,1114,2016-08-05,8,Friday
3035,2016/8/5,11,1267,1097,1,2,167,0,0,0,1267,2016-08-05 11:00:00,1100,2016-08-05,8,Friday
3036,2016/8/5,12,1283,1097,1,2,183,0,0,0,1283,2016-08-05 12:00:00,1100,2016-08-05,8,Friday
3037,2016/8/5,13,1272,1114,1,2,155,0,0,0,1272,2016-08-05 13:00:00,1117,2016-08-05,8,Friday
3038,2016/8/5,14,1266,1129,1,2,134,0,0,0,1266,2016-08-05 14:00:00,1132,2016-08-05,8,Friday
3039,2016/8/5,15,1257,1145,1,2,109,0,0,0,1257,2016-08-05 15:00:00,1148,2016-08-05,8,Friday
3040,2016/8/5,16,1245,1172,1,2,69,0,0,0,1245,2016-08-05 16:00:00,1176,2016-08-05,8,Friday
3041,2016/8/5,17,1229,1175,1,2,51,0,1,0,1229,2016-08-05 17:00:00,1178,2016-08-05,8,Friday


In [13]:
# グラフを描く
